In [17]:
# Importering av biblioteker
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output
import plotly.express as px

# Definerer dataene fra datasettet
df = pd.read_csv('komplett_data_med_utfylte_mengder.csv',parse_dates=['datetime'])



måneder_norsk = ['Januar', 'Februar', 'Mars', 'April', 'Mai', 'Juni',
                 'Juli', 'August', 'September', 'Oktober', 'November', 'Desember']
år_liste = list(range(2011, 2022)) 
måned_options = [(navn, i) for i, navn in enumerate(måneder_norsk, start=1)]



In [ ]:
# --- Globale variabler ---
dropdown_widgets = []
widgets_periode = []
output_boks = widgets.Output()
info_boks = widgets.HTML(layout=widgets.Layout(width='320px', padding='10px 20px'))
output_boks.layout = widgets.Layout(padding='30px')

# --- Overskrift ---
overskrift = widgets.HTML(
    value="<h1 style='color:#333; text-align:center;'>Interaktiv visualisering av MW-produksjon</h1>"
)

# --- Første valg: Hva ønsker du å visualisere? ---
valg_visualisering = widgets.Dropdown(
    options=["Sammenligning av tidsperioder", "Vis én tidsperiode"],
    value="Sammenligning av tidsperioder",
    description="Velg visualisering:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='420px')
)

knapp_neste = widgets.Button(description="Gå videre", button_style='success', layout=widgets.Layout(width='250px'))
knapp_bygg = widgets.Button(description="Bygg ny visualisering", button_style='success', layout=widgets.Layout(width='250px'))
knapp_start = widgets.Button(description="Start", button_style='info', layout=widgets.Layout(width='250px'))
knapp_reset = widgets.Button(description="Tilbake til start", button_style='warning', layout=widgets.Layout(width='200px'))


# --- Valg for antall perioder ---
velg_antall = widgets.Dropdown(
    options=list(range(1, 11)),
    value=2,
    description="Velg antall tidsperioder du ønsker å visualisere:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='450px')
)

visningstype = widgets.Dropdown(
    options=["Linjegraf", "Søylediagram", "Boksplott"],
    value="Linjegraf",
    description="Visningstype",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='250px')
)

# --- Funksjon for å hente og formatere data ---
def hent_data(aar, mnd, dag):
    data = df[(df["datetime"].dt.year == aar) & (df["datetime"].dt.month == mnd)].copy()
    if dag > 0:
        data = data[data["datetime"].dt.day == dag]
    label = f"{måneder_norsk[mnd-1]} {aar}" if dag == 0 else f"{måneder_norsk[mnd-1]} {aar} – dag {dag}"
    data["visningstid"] = data["datetime"].apply(lambda x: x.replace(year=2025, month=1, day=1) if dag > 0 else x.replace(year=2025, month=1))
    data["label"] = label
    return data

# --- Funksjon for å vise graf ---
def vis_graf(*args, visningstype):
    data_liste = [hent_data(*args[i:i+3]) for i in range(0, len(args), 3)]
    samlet = pd.concat(data_liste)
    dag_verdier = args[2::3]
    er_dagvisning = any(d > 0 for d in dag_verdier)
    x_label = "Tid på døgnet" if er_dagvisning else "Dag i måneden"

    # Velg riktig plott basert på visningstype
    if visningstype == "Linjegraf":
        fig = px.line(
            samlet, x="visningstid", y="MW", color="label",
            title="Visualisering av ønskede tidsperiode(r)",
            labels={"visningstid": x_label, "MW": "Megawatt (MW)", "label": "Valgte perioder"}
        )
    elif visningstype == "Søylediagram":
    # Lag en oppsummering: total MW per valgt periode
        summering = pd.DataFrame({
            "label": [d["label"].iloc[0] for d in data_liste],"MW": [d["MW"].sum() for d in data_liste]})
    
        fig = px.bar(summering, x="label", y="MW", color="label",
                     title="Søylediagram – total produksjon per valgt periode",
                     labels={"label": "Valgte perioder", "MW": "Megawatt (MW)"}
                     )
        fig.update_traces(opacity=0.9, width=0.5)

    elif visningstype == "Boksplott":
        fig = px.box(
            samlet, x="label", y="MW", color="label",
            title="Boksplott for ønskede tidsperiode(r)",
            labels={"label": "Valgte perioder", "MW": "Megawatt (MW)"}
        )

    fig.update_layout(hovermode="x unified")
    if visningstype != "Boksplott":
        fig.update_xaxes(tickformat="%H:%M" if er_dagvisning else "%d")



    # Infoboks
    totaler = [int(d["MW"].sum()) for d in data_liste]
    if len(data_liste) == 1:
        info_boks.value = f"""
            <div style='border:1px solid #ccc; border-radius:8px; background-color:#f9f9f9; padding:10px 15px;'>
              <h4>Totalt produsert MW:</h4>
              <ul>
                <li><b>{data_liste[0]['label'].iloc[0]}</b>: {totaler[0]:,} MW</li>
              </ul>
            </div>
        """
    else:
        forskjell = totaler[-1] - totaler[0]
        tegn = "+" if forskjell >= 0 else ""
        info_boks.value = f"""
            <div style='border:1px solid #ccc; border-radius:8px; background-color:#f9f9f9; padding:10px 15px;'>
              <h4>Totalt produsert MW i periodene:</h4>
              <ul>
                <li><b>{data_liste[0]['label'].iloc[0]}</b>: {totaler[0]:,} MW</li>
                <li><b>{data_liste[-1]['label'].iloc[0]}</b>: {totaler[-1]:,} MW</li>
                <li><b>Forskjellen fra periode 1 til {len(data_liste)}:</b> {tegn}{forskjell:,} MW</li>
              </ul>
            </div>
        """
    return fig

# --- Dynamisk bygging for sammenligning ---
def lag_valg_interface(antall):
    dropdown_widgets.clear()
    widgets_periode.clear()
    for i in range(antall):
        år = widgets.Dropdown(options=år_liste, value=2011, description=f"År {i+1}")
        mnd = widgets.Dropdown(options=måned_options, value=1, description=f"Måned {i+1}")
        dag = widgets.Dropdown(options=[(f"Dag {i}", i) for i in range(0, 32)], value=0, description=f"Dag {i+1}")
        dropdown_widgets.extend([år, mnd, dag])
        widgets_periode.append(widgets.VBox([år, mnd, dag]))

    rad_1 = widgets.HBox(widgets_periode)
    all_controls = widgets.VBox([rad_1, visningstype, knapp_start])

    with output_boks:
        clear_output()
        display(widgets.VBox([overskrift, all_controls]))

    info_boks.value = ""

# --- Dynamisk bygging for én periode ---
def bygg_en_periode_interface():
    dropdown_widgets.clear()
    år = widgets.Dropdown(options=år_liste, value=2011, description="År")
    mnd = widgets.Dropdown(options=måned_options, value=1, description="Måned")
    dag = widgets.Dropdown(options=[(f"Dag {i}", i) for i in range(0, 32)], value=0, description="Dag")
    dropdown_widgets.extend([år, mnd, dag])
    all_controls = widgets.VBox([
        widgets.HBox([år, mnd, dag]),
        visningstype,
        knapp_start
    ])

    with output_boks:
        clear_output(wait=True)
        display(widgets.VBox([overskrift, all_controls]))
    info_boks.value = ""


# Reset knapp
def tilbake_til_start(b):
    info_boks.value = ""

    with output_boks:
        clear_output(wait=True)
        display(widgets.VBox([overskrift, valg_visualisering, knapp_neste]))



# --- Start visualisering ---
def start_visning(b):
    args = [w.value for w in dropdown_widgets]
    fig = vis_graf(*args, visningstype=visningstype.value)

    with output_boks:
        clear_output(wait=True)
        display(overskrift)
        display(fig)  # vis plotly-fig separat
        display(info_boks)
        display(widgets.HBox([knapp_reset], layout=widgets.Layout(justify_content="flex-start")))




# --- Håndter første valg ---
def velg_visualisering(b):
    with output_boks:
        clear_output(wait=True)
        display(overskrift)
        if valg_visualisering.value == "Sammenligning av tidsperioder":
            display(widgets.VBox([velg_antall, knapp_bygg]))
        elif valg_visualisering.value == "Vis én tidsperiode":
            bygg_en_periode_interface()



# --- Koble knapper ---
knapp_bygg.on_click(lambda b: lag_valg_interface(velg_antall.value))
knapp_start.on_click(start_visning)
knapp_neste.on_click(velg_visualisering)
knapp_reset.on_click(tilbake_til_start)

# --- Startgrensesnitt ---
with output_boks:
    clear_output(wait=True)
    display(widgets.VBox([overskrift, valg_visualisering, knapp_neste]))

display(output_boks)



Output(layout=Layout(padding='30px'))